# RL methods for irrigation problems

In [3]:
%matplotlib inline
import sys, os
import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import numpy as np

data_dir = os.path.join(os.getcwd(), "data")
import pcse
print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

This notebook was built with:
python version: 3.6.10 | packaged by conda-forge | (default, Apr  6 2020, 14:40:13) 
[GCC Clang 9.0.1 ] 
PCSE version: 5.4.2


In [4]:
from pcse.fileinput import CABOFileReader
cropfile = os.path.join(data_dir, 'crop', 'sug0601.crop')
cropdata = CABOFileReader(cropfile)

In [5]:
soilfile = os.path.join(data_dir, 'soil', 'ec3.soil')
soildata = CABOFileReader(soilfile)

In [6]:
from pcse.util import WOFOST71SiteDataProvider
sitedata = WOFOST71SiteDataProvider(WAV=100, CO2=360)
print(sitedata)

{'IFUNRN': 0, 'NOTINF': 0, 'SSI': 0.0, 'SSMAX': 0.0, 'WAV': 100.0, 'SMLIM': 0.4, 'CO2': 360.0}


In [7]:
from pcse.base import ParameterProvider
parameters = ParameterProvider(cropdata=cropdata, soildata=soildata, sitedata=sitedata)

In [8]:
from pcse.fileinput import YAMLAgroManagementReader
agromanagement_file = os.path.join(data_dir, 'agro', 'sugarbeet_calendar.agro')
agromanagement = YAMLAgroManagementReader(agromanagement_file)
#print(agromanagement)

/Users/mikhailgasanov/opt/anaconda3/envs/py3_pcse/lib/python3.6/site-packages/pcse/fileinput/yaml_agro_loader.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  r = yaml.load(fp)


In [10]:
K_kg = 60
P_kg = 60
N_kg = 120
year_date=2019
yaml_agro = f"""
- {year_date}-06-01:
    CropCalendar:
        crop_name: 'sugar-beet'
        variety_name: 'sugar-beet-601'
        crop_start_date: {year_date}-06-02
        crop_start_type: emergence
        crop_end_date: {year_date}-10-15
        crop_end_type: harvest
        max_duration: 300
    TimedEvents:
    -   event_signal: apply_npk
        name:  Timed N/P/K application table
        comment: All fertilizer amounts in kg/ha
        events_table:
        - {year_date}-06-22: {{N_amount : {N_kg}, P_amount: {P_kg}, K_amount: {K_kg}}}
    StateEvents: null
"""
#agromanagement = yaml.safe_load(yaml_agro)
agromanagement = yaml.load(yaml_agro)
print(agromanagement)

[{datetime.date(2019, 6, 1): {'CropCalendar': {'crop_name': 'sugar-beet', 'variety_name': 'sugar-beet-601', 'crop_start_date': datetime.date(2019, 6, 2), 'crop_start_type': 'emergence', 'crop_end_date': datetime.date(2019, 10, 15), 'crop_end_type': 'harvest', 'max_duration': 300}, 'TimedEvents': [{'event_signal': 'apply_npk', 'name': 'Timed N/P/K application table', 'comment': 'All fertilizer amounts in kg/ha', 'events_table': [{datetime.date(2019, 6, 22): {'N_amount': 120, 'P_amount': 60, 'K_amount': 60}}]}], 'StateEvents': None}}]


/Users/mikhailgasanov/opt/anaconda3/envs/py3_pcse/lib/python3.6/site-packages/ipykernel_launcher.py:24: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


### Weather

In [ ]:
#NASA Weather system
from pcse.db import NASAPowerWeatherDataProvider
moscow_weather = NASAPowerWeatherDataProvider(55.45, 37.36)
print(moscow_weather)

### Model running

In [ ]:
from pcse.models import Wofost71_WLP_FD
%%time
wofsim = Wofost71_WLP_FD(parameters, wdp, agromanagement)
wofsim.run_till_terminate()
df_results= pd.DataFrame(wofsim.get_output())
df_results = df_results.set_index("day")
df_results.tail()

In [ ]:
summary_output = wofsim.get_summary_output()
wofsim.get_summary_output()
msg = "Reached maturity at {DOM} with total biomass {TAGP} kg/ha "\
"and a yield of {TWSO} kg/ha."
print(msg.format(**summary_output[0]))

## Agromanagment

In [2]:
K_kg = 60
P_kg = 60
N_kg = 120
year_date=2019
yaml_agro = f"""
- {year_date}-06-01:
    CropCalendar:
        crop_name: 'sugar-beet'
        variety_name: 'sugar-beet-601'
        crop_start_date: {year_date}-06-02
        crop_start_type: emergence
        crop_end_date: {year_date}-10-15
        crop_end_type: harvest
        max_duration: 300
    TimedEvents:
    -   event_signal: apply_npk
        name:  Timed N/P/K application table
        comment: All fertilizer amounts in kg/ha
        events_table:
        - {year_date}-06-22: {{N_amount : {N_kg}, P_amount: {P_kg}, K_amount: {K_kg}}}
    StateEvents: null
"""
agromanagement = yaml.load(yaml_agro)

NameError: name 'yaml' is not defined

In [ ]:
def climate_change(year):
    K_kg = 60
    P_kg = 60
    N_kg = 120

    year_date=year
    yaml_agro = f"""
    - {year_date}-05-01:
        CropCalendar:
            crop_name: 'sugar-beet'
            variety_name: 'sugar-beet-601'
            crop_start_date: {year_date}-05-02
            crop_start_type: sowing
            crop_end_date: 
            crop_end_type: maturity
            max_duration: 250
        TimedEvents:
        -   event_signal: apply_npk
            name:  Timed N/P/K application table
            comment: All fertilizer amounts in kg/ha
            events_table:
            - {year_date}-06-22: {{N_amount : {N_kg}, P_amount: {P_kg}, K_amount: {K_kg}}}
        StateEvents: null
    """
    agromanagement = yaml.safe_load(yaml_agro)
    #crop_end_date: {year_date_1}-11-15

    parameters = ParameterProvider(cropdata=cropdata, soildata=soildata, sitedata=sitedata)
    wofsim = Wofost71_WLP_FD(parameters, moscow_weather, agromanagement)
    wofsim.run_till_terminate()
    #df_results = pd.DataFrame(wofsim.get_output())
    #df_results = df_results.set_index("day")
    #df_results.tail()
    summary_output = wofsim.get_summary_output()
    yield_list.append(summary_output[0]['TWSO']) #тут можно и другие параметры сохранять
    LAE_list.append(summary_output[0]['LAE']) #тут можно и другие параметры сохранять